In [8]:
import numpy as np
import json
import warnings

import h5py
from keras.models import model_from_json

from matplotlib import pyplot as plt

warnings.filterwarnings("ignore")

size_title = 18
size_label = 14
n_pred = 2

path_data_dict = "data/generated_files/data_dict.txt"
path_inverted_wt = "data/generated_files/inverted_weights.txt"
path_usage_wt = "data/generated_files/usage_prediction.txt"
path_class_wt = "data/generated_files/class_weights.txt"
path_test_data = "data/generated_files/test_data.txt"
model_path = "data/generated_files/trained_model.hdf5"

def read_file(file_path):
    with open(file_path, 'r') as data_file:
        data = json.loads(data_file.read())
    return data

def create_model(model_path):
    trained_model = h5py.File(model_path, 'r')
    model_config = json.loads(trained_model.get('model_config').value)
    loaded_model = model_from_json(model_config)
    dictionary = json.loads(trained_model.get('data_dictionary').value)
    compatibile_tools = json.loads(trained_model.get('compatible_tools').value)
    reverse_dictionary = dict((v, k) for k, v in dictionary.items())
    model_weights = list()
    weight_ctr = 0
    while True:
        try:
            d_key = "weight_" + str(weight_ctr)
            weights = trained_model.get(d_key).value
            model_weights.append(weights)
            weight_ctr += 1
        except Exception as exception:
            break
    # set the model weights
    loaded_model.set_weights(model_weights)
    return loaded_model, dictionary, reverse_dictionary, compatibile_tools

model, dictionary, reverse_dictionary, compatibile_tools = create_model(model_path)


In [28]:
def verify_model(model, tool_sequence, labels, dictionary, reverse_dictionary, compatible_tools, topk=3, max_seq_len=25):
    
    sample = np.zeros(max_seq_len)
    tl_seq = tool_sequence.split(",")
    last_tool_name = reverse_dictionary[int(tl_seq[-1])]
    last_compatible_tools = compatible_tools[last_tool_name]

    for idx, tool_id in enumerate(tool_sequence):
        print(tool_id)
        sample[-(idx + 1)] = int(tool_id)
        sample = np.reshape(sample, (1, max_seq_len))

        # predict next tools for a test path
        prediction = model.predict(sample, verbose=0)
        prediction = np.reshape(prediction, (prediction.shape[1],))
        prediction_pos = np.argsort(prediction, axis=-1)

        # get topk prediction
        topk_prediction_pos = prediction_pos[-topk:]
        topk_prediction_val = [np.round(prediction[pos] * 100, 2) for pos in topk_prediction_pos]
        
        print(topk_prediction_pos, topk_prediction_val)

        # read tool names using reverse dictionary
        pred_tool_ids = [reverse_dictionary[int(tool_pos)] for tool_pos in topk_prediction_pos]
        actual_next_tool_ids = list(set(pred_tool_ids).intersection(set(last_compatible_tools.split(","))))
        
        print(pred_tool_ids, actual_next_tool_ids)

        pred_tool_ids_sorted = dict()
        for (tool_pos, tool_pred_val) in zip(topk_prediction_pos, topk_prediction_val):
            tool_name = reverse_dictionary[int(tool_pos)]
            if tool_name in actual_next_tool_ids:
                pred_tool_ids_sorted[tool_name] = tool_pred_val
        print(pred_tool_ids_sorted)
        # find false positives
        #false_positives = [ tool_name for tool_name in top_predicted_next_tool_names if tool_name not in actual_next_tool_names ]
        #absolute_precision = 1 - ( len( false_positives ) / float( len( actual_classes_pos ) ) )
        #ave_abs_precision.append(absolute_precision)
    #mean_precision = np.mean(ave_abs_precision)
    #print("Absolute precision on test data using current model is: %0.6f" % mean_precision)
    #return mean_precision

def get_predictions(model, dictionary, reverse_dictionary, compatibile_tools):
    t_data = read_file(path_test_data)
    for ph, cl in t_data.items():
        print(ph, cl)
        verify_model(model, ph, cl, dictionary, reverse_dictionary, compatibile_tools)
        #break

get_predictions(model, dictionary, reverse_dictionary, compatibile_tools)

615,841,471,775,439,24,489,497,457,464,192,875,816,118 680,244,231,426
6
[489 190 841] [3.33, 3.78, 6.96]
['Filter1', 'htseq_count', 'fastqc'] ['Filter1', 'fastqc']
{'Filter1': 3.33, 'fastqc': 6.96}
1


IndexError: index -2 is out of bounds for axis 0 with size 1

NameError: name 'topk_prediction_pos' is not defined